In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import sys
import itertools

In [ ]:
def load_data(csv_file, path_column='path', question_column='question'):
    """
    Legge il CSV e ritorna una lista di path e una lista di domande.
    I path sono formattati come stringhe "decision -> decision -> ...".
    """
    df = pd.read_csv(csv_file)
    # Estrae i path e li suddivide in liste di decisioni
    paths = df[path_column].dropna().tolist()
    paths = [[d.strip() for d in p.split('->')] for p in paths]
    
    # Estrae le domande se presenti
    questions = []
    if question_column in df.columns:
        questions = df[question_column].dropna().tolist()
        
    return paths, questions

def get_unique_decisions(paths):
    """
    Estrae tutte le decisioni uniche dai path.
    """
    decisions = set()
    for path in paths:
        decisions.update(path)
    return list(decisions)

def cluster_decisions(decisions, model, similarity_threshold=0.8):
    """
    Crea le embedding delle decisioni e le raggruppa in cluster usando AgglomerativeClustering.
    La soglia di similarità (coseno) viene usata per definire la distanza (1 - similarità).
    """
    # Ottieni le embedding per ciascuna decisione
    embeddings = model.encode(decisions, convert_to_numpy=True)
    # Calcola la distanza: 1 - cos_sim
    distance_threshold = 1 - similarity_threshold

    # Agglomerative clustering con distanza basata su similarità coseno
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold, metric='cosine', linkage='average')
    labels = clustering.fit_predict(embeddings)
    
    # Raggruppa le decisioni per cluster
    clusters = {}
    for decision, label in zip(decisions, labels):
        clusters.setdefault(label, []).append(decision)
    return clusters, labels, embeddings

def get_cluster_representatives(clusters, model):
    """
    Per ogni cluster, seleziona una decisione rappresentativa (ad esempio, quella più vicina al centroide).
    """
    representatives = {}
    for label, items in clusters.items():
        # Ottieni le embedding per le decisioni del cluster
        item_embeddings = model.encode(items, convert_to_numpy=True)
        # Calcola il centroide del cluster
        centroid = np.mean(item_embeddings, axis=0)
        # Seleziona la decisione più vicina al centroide
        distances = np.linalg.norm(item_embeddings - centroid, axis=1)
        rep = items[np.argmin(distances)]
        representatives[label] = rep
    return representatives

def build_path_graph(paths, decision_to_cluster):
    """
    Crea un grafo diretto basato sui path originali.
    Per ogni path, viene mappata la sequenza di decisioni nel rispettivo rappresentante (cluster)
    e vengono aggiunti gli archi con peso incrementale.
    """
    G = nx.Graph()
    for path in paths:
        # Mappa ogni decisione al suo rappresentante (cluster)
        aggregated_path = [decision_to_cluster[d] for d in path]
        # Aggiungi nodi e archi (incrementando il peso se l'arco è già presente)
        for i in range(len(aggregated_path)-1):
            u = aggregated_path[i]
            v = aggregated_path[i+1]
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)
    return G

def build_knn_graph(cluster_embeddings, representatives, k=5):
    """
    Costruisce un grafo non diretto (KNN graph) usando le embedding dei rappresentanti dei cluster.
    Vengono collegati ciascun nodo ai suoi k vicini (saltando il nodo stesso).
    """
    # Prepara la lista di rappresentanti e le relative embedding
    rep_texts = list(representatives.values())
    rep_embeddings = []
    # Mappa da etichetta a rappresentante
    rep_label = {}
    for label, rep in representatives.items():
        rep_label[rep] = label
        rep_embeddings.append(cluster_embeddings[label])
    rep_embeddings = np.vstack(rep_embeddings)
    
    # Trova i k vicini usando NearestNeighbors (metric 'cosine')
    nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(rep_embeddings)
    distances, indices = nbrs.kneighbors(rep_embeddings)
    
    G_knn = nx.Graph()
    for i, (dists, idxs) in enumerate(zip(distances, indices)):
        for j, idx in enumerate(idxs[1:]):  # ignora il nodo stesso
            u = rep_texts[i]
            v = rep_texts[idx]
            # La similarità coseno = 1 - distanza
            weight = 1 - dists[j]
            G_knn.add_edge(u, v, weight=weight)
    return G_knn

def retrieve_similar_nodes(questions, representatives, cluster_embeddings, model, k=3):
    """
    Per ciascuna domanda, calcola l'embedding e ne ricava i k nodi più simili basandosi sulla similarità coseno.
    Restituisce un dizionario: domanda -> lista di (nodo, similarità).
    """
    # Prepara lista dei rappresentanti e delle loro embedding
    rep_texts = list(representatives.values())
    rep_labels = list(representatives.keys())
    rep_emb_list = [cluster_embeddings[label] for label in rep_labels]
    rep_embs = np.vstack(rep_emb_list)
    
    results = {}
    for question in questions:
        q_emb = model.encode([question], convert_to_numpy=True)
        sims = cosine_similarity(q_emb, rep_embs)[0]
        # Ottieni gli indici dei k nodi più simili ordinati per similarità decrescente
        top_indices = sims.argsort()[-k:][::-1]
        similar_nodes = [(rep_texts[i], sims[i]) for i in top_indices]
        results[question] = similar_nodes
    return results

def check_edges_between_nodes(nodes, graph):
    """
    Dati una lista di nodi, controlla se esistono edge (in una direzione o nell'altra) nel grafo.
    Restituisce una lista di tuple (u, v, data) per ogni edge trovato.
    """
    edges_found = []
    # Controlla ogni coppia di nodi
    for u, v in itertools.combinations(nodes, 2):
        # Per grafi misti (diretto e indiretto) controlliamo in entrambe le direzioni
        if graph.has_edge(u, v):
            edges_found.append((u, v, graph.get_edge_data(u, v)))
        if graph.has_edge(v, u):
            edges_found.append((v, u, graph.get_edge_data(v, u)))
    return edges_found

In [ ]:
 # Carica i path dal file CSV
paths, questions = load_data("/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full_balanced.csv")

# Estrai tutte le decisioni uniche
unique_decisions = get_unique_decisions(paths)

# Inizializza il modello di embedding
model = SentenceTransformer('all-mpnet-base-v2')

# Esegui il clustering delle decisioni per aggregare quelle simili
clusters, labels, embeddings = cluster_decisions(unique_decisions, model, similarity_threshold=0.7)

# Seleziona un rappresentante per ogni cluster
representatives = get_cluster_representatives(clusters, model)

# Crea una mappa: decisione originale -> rappresentante del cluster
decision_to_cluster = {}
for decision, label in zip(unique_decisions, labels):
    decision_to_cluster[decision] = representatives[label]

# Costruisci il grafo basato sui path (grafo diretto) mantenendo gli edge originali aggregati
path_graph = build_path_graph(paths, decision_to_cluster)
print("Nodi del Path Graph:")
print(path_graph.number_of_nodes())
print("\nArchi del Path Graph:")
print(path_graph.number_of_edges())

# Prepara le embedding dei rappresentanti (una per ogni cluster)
cluster_embeddings = {}
for label, items in clusters.items():
    # Utilizziamo la embedding del rappresentante come embedding del cluster
    idx = unique_decisions.index(representatives[label])
    cluster_embeddings[label] = embeddings[idx]

# Costruisci il grafo KNN usando le embedding dei rappresentanti
knn_graph = build_knn_graph(cluster_embeddings, representatives, k=5)
print("\nNodi del KNN Graph:")
print(knn_graph.number_of_nodes())
print("\nArchi del KNN Graph:")
print(knn_graph.number_of_edges())

# Unisci i due grafi: manteniamo sia gli edge derivanti dai path originali che quelli dal KNN graph
final_graph = nx.compose(path_graph, knn_graph)
print("\nNodi del Grafo Finale:")
print(final_graph.number_of_nodes())
print("\nArchi del Grafo Finale:")
print(final_graph.number_of_edges())
# (Opzionale) Salva o visualizza il grafo finale, ad esempio:
# nx.write_gpickle(final_graph, "final_graph.gpickle")

In [ ]:
# Se sono presenti domande, effettua il retrieval dei nodi più simili per ciascuna
if questions:
    retrieval = retrieve_similar_nodes(questions, representatives, cluster_embeddings, model, k=3)
    print("\nRisultati del retrieval per ogni domanda:")
    for question, similar_nodes in retrieval.items():
        print(f"\nDomanda: {question}")
        nodes_retrieved = [node for node, sim in similar_nodes]
        for node, sim in similar_nodes:
            print(f"  Nodo: {node} - Similarità: {sim:.4f}")
        edges_between = check_edges_between_nodes(nodes_retrieved, final_graph)
        if edges_between:
            print("  Gli edge che collegano i nodi sono:")
            for u, v, data in edges_between:
                print(f"    {u} -> {v} con attributi {data}")
        else:
            print("  Nessun edge diretto trovato tra i nodi recuperati.")